In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from tqdm.auto import tqdm
import os

import numpy as np
import pandas as pd 

import matplotlib.pyplot as plt
import seaborn as sns 

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split , StratifiedKFold


import tensorflow as tf 
import tensorflow.keras.backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input,Dense, LSTM, RNN, Bidirectional, GlobalAveragePooling2D , Dropout

from transformers import TFAutoModel , AutoTokenizer

In [ ]:
df_train = pd.read_csv('/kaggle/input/iiitd-abuse-detection-challenge/dummy_data.csv')
df_train

In [ ]:
 hf_path = "google/muril-base-cased" 
max_len  =64

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(hf_path)

In [ ]:
transformer_model = TFAutoModel.from_pretrained(hf_path)

In [ ]:
input_id_layer = Input(shape=(max_len,) ,dtype = tf.int32 , name = 'input_ids')
attention_mask_layer = Input(shape=(max_len,) , dtype = tf.int32 , name = 'attention_mask')

transformer = transformer_model(input_ids = input_id_layer , attention_mask = attention_mask_layer)[0]
transformer_output = transformer[:,0,:]

x = Dropout(0.2)(transformer_output)
predictions = Dense(1, activation = "sigmoid")(x)

model = Model(inputs=[input_id_layer , attention_mask_layer], outputs = predictions)
model.compile(
      optimizer = Adam(learning_rate= 1e-5),
      metrics = 'AUC',
      loss = 'binary_crossentropy'
  )

model.summary()

In [ ]:
model.load_weights('/kaggle/input/tpu-muril-training/result/muril_fold5.h5')

In [ ]:
text="goodbay"


encs = tokenizer(text,
                    max_length = max_len,
                    padding= 'max_length',
                    truncation=True,
                    return_tensors ='tf'
                    )
input_ids  = encs['input_ids']
attention_mask = encs['attention_mask']
# Preprocess input


inputs  = [input_ids , attention_mask]
# Make prediction
prediction = model(inputs)[0]

(prediction)>0.5



In [ ]:
attention_mask

In [ ]:
model.save('Muril_best_model1.h5')

In [ ]:
import transformers
loaded_model = tf.keras.models.load_model('/kaggle/working/Muril_best_model1.h5', custom_objects={"TFBertModel": transformers.TFBertModel})
